## 多线程的超时

In [1]:
using System.Threading;
var thread = new Thread(Foo);

thread.Start();

if(!thread.Join(TimeSpan.FromMilliseconds(2000)))
{
    thread.Interrupt();
}

Console.WriteLine("Done.");

void Foo()
{
    try
    {
        Console.WriteLine("Foo start...");
        Thread.Sleep(500);
        Console.WriteLine("Foo end...");
    }
    catch(ThreadInterruptedException)
    {
        Console.WriteLine("Foo interrupted...");
    }
}

Foo start...
Foo end...
Done.


## 异步编程的超时

In [6]:
var cts = new CancellationTokenSource();
var task = FooAsync(cts.Token);
var completedTask = await Task.WhenAny(task, Task.Delay(2000));
if(completedTask != task)
{
    cts.Cancel();
    await task;
    Console.WriteLine("Timeout...");
}

//await Task.Delay(4000);
Console.WriteLine("Done.");



async Task FooAsync(CancellationToken token)
{
    try
    {
        Console.WriteLine("Foo start...");
        await Task.Delay(5000,token);
        Console.WriteLine("Foo end...");
    }
    catch(OperationCanceledException)
    {
        Console.WriteLine("Foo interrupted...");
    }
}

Foo start...
Foo interrupted...
Timeout...
Done.


In [9]:
//public static class AsyncExceptions;
public static async Task TimeoutAfter(this Task task, TimeSpan timeout)
{
    using var cts = new CancellationTokenSource();
    var completedTask = await Task.WhenAny(task,Task.Delay(timeout,cts.Token));
    if(completedTask != task)
    {
        cts.Cancel();
        throw new TimeoutException();
    }
    await task;
}

In [8]:
try
{
    await FooAsync(CancellationToken.None).TimeoutAfter(TimeSpan.FromSeconds(3));
    Console.WriteLine("Success!");
}
catch(TimeoutException)
{
    Console.WriteLine("Timeout!");
}
Console.WriteLine("Done!");

Foo start...
Timeout!
Done!


In [13]:
var cts = new CancellationTokenSource();
try
{
    //异步任务超时,.net6以后提供
    await FooAsync(cts.Token).WaitAsync(TimeSpan.FromSeconds(2));
    Console.WriteLine("Success!");
}
catch(TimeoutException)
{
    cts.Cancel();
    Console.WriteLine("Timeout!");
}
finally
{
    cts.Dispose();
}
Console.WriteLine("Done!");

Foo start...
Timeout!
Done!
Foo interrupted...
